## 2a. Evidence - Compliance - Position

Evidence collected in this section checks for position complience in the Gradient Climber Example

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [1]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from demo.GradientClimber.session import *

Creating initial custom lists at URI: local:///Users/jhansen/continuum/mlte/demo/GradientClimber/../store
Loaded 7 qa_categories for initial list
Loaded 30 quality_attributes for initial list
Creating sample catalog at URI: StoreType.LOCAL_FILESYSTEM:local:///Users/jhansen/continuum/mlte/demo/GradientClimber/../store
Loading sample catalog entries.
Loaded 9 entries for sample catalog.


In [2]:
import numpy as np
import gymnasium as gym

In [3]:
env = gym.make('MountainCar-v0',render_mode="rgb_array")
state, info = env.reset()

In [4]:
# Discretize the state space (position, velocity)
position_bins = np.linspace(-1.2, 0.6, 20)
velocity_bins = np.linspace(-0.07, 0.07, 20)

# Q-table initialization
q_table = np.load("mountain_car.npy")

# Discretize the continuous state (position and velocity)
def discretize_state(state):
    position, velocity = state
    position_idx = np.digitize(position, position_bins) - 1  # Position bin index
    velocity_idx = np.digitize(velocity, velocity_bins) - 1  # Velocity bin index
    return position_idx, velocity_idx

# Epsilon-greedy action selection
def choose_action(state):
    position_idx, velocity_idx = discretize_state(state)
    return np.argmax(q_table[position_idx, velocity_idx])

## Compliance

In [5]:
def evaluate_action(state,action):
    "Return 1 if this is the expected action, return 0 if it is the wrong move, and -1 as an error condition"
    position, velocity = state
    if (position >= -1.2) and (position <= 0.6):
        return np.bool(True)
    else:
        return np.bool(False)

In [6]:
def test_position_compliance():
    done = False
    states = [] 
    trials=100
    compliance_evidence = []

    for i in range(trials):
        test_results = []
        state,info = env.reset()
        done = False
        actions=[]

        while not done:
            # Random action selection
            action = choose_action(state)
            states.append(state)
            # Take the action and get the next state, reward, done flag, and info
            next_state, reward, done, truncated, info = env.step(action)

            result = evaluate_action(state,action)
            if result==True:
                test_results.append(1)
            elif result==False:
                test_results.append(0)
            # Update the state for the next iteration
            state = next_state
            
        frac_in_bounds = np.sum(np.array(test_results)==1)/len(test_results)
        print(f"Completed trial {i}: {frac_in_bounds}")
        compliance_evidence += [frac_in_bounds] 

    return compliance_evidence


In [7]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the TestSuite.
position_compliance_measurement = ExternalMeasurement(
    "gradient climber position accuracy", Array, test_position_compliance
)
evidence = position_compliance_measurement.evaluate()

# Inspect value
print(evidence)

# Save to artifact store
evidence.save(force=True,parents=True)

Completed trial 0: 1.0
Completed trial 1: 1.0
Completed trial 2: 1.0
Completed trial 3: 1.0
Completed trial 4: 1.0
Completed trial 5: 1.0
Completed trial 6: 1.0
Completed trial 7: 1.0
Completed trial 8: 1.0
Completed trial 9: 1.0
Completed trial 10: 1.0
Completed trial 11: 1.0
Completed trial 12: 1.0
Completed trial 13: 1.0
Completed trial 14: 1.0
Completed trial 15: 1.0
Completed trial 16: 1.0
Completed trial 17: 1.0
Completed trial 18: 1.0
Completed trial 19: 1.0
Completed trial 20: 1.0
Completed trial 21: 1.0
Completed trial 22: 1.0
Completed trial 23: 1.0
Completed trial 24: 1.0
Completed trial 25: 1.0
Completed trial 26: 1.0
Completed trial 27: 1.0
Completed trial 28: 1.0
Completed trial 29: 1.0
Completed trial 30: 1.0
Completed trial 31: 1.0
Completed trial 32: 1.0
Completed trial 33: 1.0
Completed trial 34: 1.0
Completed trial 35: 1.0
Completed trial 36: 1.0
Completed trial 37: 1.0
Completed trial 38: 1.0
Completed trial 39: 1.0
Completed trial 40: 1.0
Completed trial 41: 1.0
Co

ArtifactModel(header=ArtifactHeaderModel(identifier='evidence.gradient climber position accuracy', type='evidence', timestamp=1760129557, creator=None, level='version'), body=EvidenceModel(artifact_type=<ArtifactType.EVIDENCE: 'evidence'>, metadata=EvidenceMetadata(test_case_id='gradient climber position accuracy', measurement=MeasurementMetadata(measurement_class='mlte.measurement.external_measurement.ExternalMeasurement', output_class='mlte.evidence.types.array.Array', additional_data={'function': '__main__.test_position_compliance'})), evidence_class='mlte.evidence.types.array.Array', value=ArrayValueModel(evidence_type=<EvidenceType.ARRAY: 'array'>, data=[np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float6